In [89]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas_ta as ta
#Data Import
import yfinance as yf
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

In [90]:
# Data Download 
ticker_symbol = '^NSEI'
df = yf.download(ticker_symbol, period='max', progress=False)

In [91]:
df.head()

Price,Close,High,Low,Open,Volume
Ticker,^NSEI,^NSEI,^NSEI,^NSEI,^NSEI
Date,,,,,
2007-09-17,4494.649902,4549.049805,4482.850098,4518.450195,0
2007-09-18,4546.200195,4551.799805,4481.549805,4494.100098,0
2007-09-19,4732.350098,4739.000000,4550.250000,4550.250000,0
2007-09-20,4747.549805,4760.850098,4721.149902,4734.850098,0
2007-09-21,4837.549805,4855.700195,4733.700195,4752.950195,0


In [92]:
## Copy of Dataframe to work in
df_modified = df.copy()

In [93]:
# We flatten the columns to ensure they are simple and single-level.
if isinstance(df_modified.columns, pd.MultiIndex):
    df_modified.columns = df_modified.columns.get_level_values(0)
else:
    print("Columns are already single-level. No flattening needed.")

In [94]:
# Add all engineered features to the modified DataFrame

df_modified['Daily Return'] = df_modified['Close'].pct_change()
df_modified['MA50'] = df_modified['Close'].rolling(window=50).mean()
df_modified['MA200'] = df_modified['Close'].rolling(window=200).mean()
df_modified['RSI'] = ta.rsi(df_modified['Close'], length=14)


In [95]:
df_modified.tail()

Price,Close,High,Low,Open,Volume,Daily Return,MA50,MA200,RSI
Date,,,,,,,,,
2025-07-31,24768.349609,24956.500000,24635.000000,24642.250000,346300,-0.003488,25057.622070,24053.279277,40.914833
2025-08-01,24565.349609,24784.150391,24535.050781,24734.900391,263400,-0.008196,25051.866055,24050.466279,36.006905
2025-08-04,24722.750000,24736.250000,24554.000000,24596.050781,259300,0.006407,25046.298047,24048.793281,41.833134
2025-08-05,24649.550781,24733.099609,24590.300781,24720.250000,251800,-0.002961,25042.765078,24047.184531,40.008836
2025-08-06,24574.199219,24671.400391,24539.199219,24641.349609,0,-0.003057,25039.200078,24046.306279,38.163831


In [96]:
# Check how many missing (NaN) values are in each column

df_modified.isnull().sum()

Price
Close             0
High              0
Low               0
Open              0
Volume            0
Daily Return      1
MA50             49
MA200           199
RSI              14
dtype: int64

In [97]:
# shape before dropping nan value
df_modified.shape

(4388, 9)

In [98]:
# Drop rows with any NaN values directly from df_modified
df_modified.dropna(inplace=True)

In [99]:
# shape after dropping 
df_modified.shape

(4189, 9)

#### Preparing X and Y variables